# Build custom container for Vertex Prediction

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/02-perarm-features-bandit


In [2]:
# PREFIX = 'mabv1'
VERSION        = "v2"                       # TODO
PREFIX         = f'rec-bandits-{VERSION}'   # TODO

print(f"PREFIX: {PREFIX}")

PREFIX: rec-bandits-v2


In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "rec-bandits-v2"
VERSION                  = "v2"

BUCKET_NAME              = "rec-bandits-v2-hybrid-vertex-bucket"
BUCKET_URI               = "gs://rec-bandits-v2-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://rec-bandits-v2-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"
DATA_PATH_KFP_DEMO       = "gs://rec-bandits-v2-hybrid-vertex-bucket/data/kfp_demo_data/u.data"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_NAME    = "mvlens_rec_bandits_v2"
BIGQUERY_TABLE_NA

In [4]:
# TODO incorporate to 00-env-setup
DOCKERNAME_02_PRED = 'Dockerfile_predict_mab_02e'
IMAGE_NAME_02_PRED = "pred-perarm-feats-02e"
IMAGE_URI_02_PRED  = f"gcr.io/hybrid-vertex/{IMAGE_NAME_02_PRED}"
REMOTE_IMAGE_NAME  = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE_NAME_02_PRED}"

print(f"DOCKERNAME_02_PRED = {DOCKERNAME_02_PRED}")
print(f"REPOSITORY         = {REPOSITORY}")
print(f"IMAGE_NAME_02_PRED = {IMAGE_NAME_02_PRED}")
print(f"IMAGE_URI_02_PRED  = {IMAGE_URI_02_PRED}")
print(f"REMOTE_IMAGE_NAME  = {REMOTE_IMAGE_NAME}")

DOCKERNAME_02_PRED = Dockerfile_predict_mab_02e
REPOSITORY         = rl-movielens-rec-bandits-v2
IMAGE_NAME_02_PRED = pred-perarm-feats-02e
IMAGE_URI_02_PRED  = gcr.io/hybrid-vertex/pred-perarm-feats-02e
REMOTE_IMAGE_NAME  = us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/pred-perarm-feats-02e


## Create Artifact Repository
If you don't have an existing artifact repository, create one using the gcloud command below

In [5]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

In [6]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

## Set gcloudignore

In [7]:
! gcloud config set gcloudignore/enabled true

Updated property [gcloudignore/enabled].


run this in terminal from root to clear `__pycache__` files...

In [8]:
# find . | grep -E "(/__pycache__$|\.pyc$|\.pyo$)" | xargs rm -rf

In [9]:
# !tree src -L 2 -I "archive"

## Create serving app

In [10]:
!ls src

archive  per_arm_rl  perarm_features  pred  ranking  serve


In [11]:
SERVING_DIR = "serve"
SERVING_APP_DIR = "app"

# ! rm -f $SERVING_APP_DIR

In [12]:
# %mkdir src/$SERVING_APP_DIR

# Make folder for Python training script
! rm -rf src/$SERVING_DIR
! mkdir src/$SERVING_DIR

# Make the predictor subfolder
! mkdir src/$SERVING_DIR/$SERVING_APP_DIR

In [13]:
! cp -r ./src/pred/* ./src/$SERVING_DIR/$SERVING_APP_DIR/

In [14]:
!ls src

archive  per_arm_rl  perarm_features  pred  ranking  serve


In [15]:
! ls ./src/$SERVING_DIR/$SERVING_APP_DIR

__init__.py	emb_features_pred.py  prestart.sh	vocab_dict.pkl
data_config.py	main.py		      requirements.txt


overwrite `.gcloudignore`...

In [16]:
# .ipynb_checkpoints/*
# *cpython-37.pyc
# **.cpython-310.pyc

# src/per_arm_rl/*
# src/perarm_features/*
# */__pycache__/*

In [18]:
%%writefile .gcloudignore
.gcloudignore
.ipynb_checkpoints/*
*.cpython-310.pyc
WIP/*
imgs/*
learning/*
*.png
*.ipynb
.git
.github
.gitignore
.DS_Store
*.md
*.tfrecord
src/ranking/*
src/archive/*
04-pipelines/*
01-baseline-perarm-bandit/*
Dockerfile_perarm_feats
Dockerfile_perarm_feats_tpu
Dockerfile_predict_mab_02
Dockerfile_train_my_perarm_env
vertex_env/*
credentials.json

Overwriting .gcloudignore


In [19]:
!gcloud meta list-files-for-upload

requirements.txt
pred_instances.json
Dockerfile_predict_mab_02e
cloudbuild.yaml
02-perarm-features-bandit/vocab_dict.pkl
src/pred/emb_features_pred.py
src/pred/requirements.txt
src/pred/__init__.py
src/pred/vocab_dict.pkl
src/pred/prestart.sh
src/pred/main.py
src/pred/data_config.py
src/per_arm_rl/utils_config.py
src/per_arm_rl/perarm_task.py
src/per_arm_rl/__init__.py
src/per_arm_rl/my_per_arm_py_env.py
src/per_arm_rl/policy_util.py
src/per_arm_rl/train_utils.py
src/per_arm_rl/trainer_baseline.py
src/per_arm_rl/data_utils.py
src/per_arm_rl/data_config.py
src/serve/app/emb_features_pred.py
src/serve/app/requirements.txt
src/serve/app/__init__.py
src/serve/app/vocab_dict.pkl
src/serve/app/prestart.sh
src/serve/app/main.py
src/serve/app/data_config.py
src/perarm_features/train_perarm.py
src/perarm_features/reward_factory.py
src/perarm_features/emb_features.py
src/perarm_features/agent_factory.py
src/perarm_features/__init__.py
src/perarm_features/trainer_common.py
src/perarm_features/tas

In [21]:
!tree src -L 3 -I "archive"

src
├── per_arm_rl
│   ├── __init__.py
│   ├── data_config.py
│   ├── data_utils.py
│   ├── my_per_arm_py_env.py
│   ├── perarm_task.py
│   ├── policy_util.py
│   ├── train_utils.py
│   ├── trainer_baseline.py
│   └── utils_config.py
├── perarm_features
│   ├── __init__.py
│   ├── agent_factory.py
│   ├── emb_features.py
│   ├── eval_perarm.py
│   ├── ranking_bandit_policy.py
│   ├── reward_factory.py
│   ├── task.py
│   ├── train_perarm.py
│   └── trainer_common.py
├── pred
│   ├── __init__.py
│   ├── data_config.py
│   ├── emb_features_pred.py
│   ├── main.py
│   ├── prestart.sh
│   ├── requirements.txt
│   └── vocab_dict.pkl
├── ranking
│   ├── jw_perarm_mv_env.py
│   ├── jw_perarm_mv_env_tf.py
│   ├── ranking_trainer.py
│   ├── repeated_feature_perarm_network.py
│   ├── stationary_stochastic_repeated_feature_py_environment.py
│   └── tmp_cascading_bandit_perarm_env.py
└── serve
    └── app
        ├── __init__.py
        ├── data_config.py
        ├── emb_features_pred.py
        ├

## Build Dockerfile

In [22]:
# COPY src/prediction /app
# COPY src/perarm_features /app/perarm_features

# CMD ["sh", "-c", "uvicorn src.prediction.main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

# ENV PYTHONUNBUFFERED True

# ARG FOO=bar
# ARG FOO
# RUN echo "these steps will be executed again if the value of FOO changes"

# COPY src/perarm_features $APP_HOME/src/perarm_features
# COPY src/per_arm_rl $APP_HOME/src/per_arm_rl
# COPY src/prediction $APP_HOME/src/prediction

# ADD src $APP_HOME/src

# COPY src/pred $APP_HOME

# RUN pip3 install --no-cache-dir -r $APP_HOME/requirements.txt
# CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port 7080"]

In [23]:
dockerfile = f'''
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.10
# FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-13.py310
# FROM tensorflow/tensorflow:2.14.0

# Allow statements and log messages to immediately appear in the logs.
ENV PYTHONUNBUFFERED True

ENV APP_HOME /app

WORKDIR $APP_HOME

COPY ./app /$APP_HOME

RUN ls $APP_HOME

RUN pip3 install --upgrade pip && pip3 install --no-cache-dir -r $APP_HOME/requirements.txt

EXPOSE 80

CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]
'''
print(dockerfile)


FROM tiangolo/uvicorn-gunicorn-fastapi:python3.10
# FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-13.py310
# FROM tensorflow/tensorflow:2.14.0

# Allow statements and log messages to immediately appear in the logs.
ENV PYTHONUNBUFFERED True

ENV APP_HOME /app

WORKDIR $APP_HOME

COPY ./app /$APP_HOME

RUN ls $APP_HOME

RUN pip3 install --upgrade pip && pip3 install --no-cache-dir -r $APP_HOME/requirements.txt

EXPOSE 80

CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]



In [24]:
with open(f'src/{SERVING_DIR}/{DOCKERNAME_02_PRED}', 'w') as f:
    f.write(dockerfile)

In [25]:
# %%bash -s $MODEL_DIR 

# MODEL_DIR=$1

# mkdir -p ./serve/model/
# gsutil cp -r ${MODEL_DIR} ./serve/model/ 

# cat > ./serve/Dockerfile <<EOF
# FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

# COPY ./app /app
# COPY ./model /
# COPY requirements.txt requirements.txt

# RUN pip install -r requirements.txt

# EXPOSE 7080
# CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "7080"]

# EOF

In [26]:
# image definitions for training
MACHINE_TYPE            ='e2-highcpu-32'
FILE_LOCATION           = f"./src/{SERVING_DIR}" # './src'

print(f"FILE_LOCATION : {FILE_LOCATION}")
print(f"MACHINE_TYPE  : {MACHINE_TYPE}")

FILE_LOCATION : ./src/serve
MACHINE_TYPE  : e2-highcpu-32


In [27]:
! gcloud builds submit --config ./cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME_02_PRED,_IMAGE_URI=$IMAGE_URI_02_PRED,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 39 file(s) totalling 820.8 KiB before compression.
Uploading tarball of [.] to [gs://hybrid-vertex_cloudbuild/source/1700046469.187503-dcbf29813b914ce4b252c64e7e879c14.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/hybrid-vertex/locations/global/builds/c9f3737a-5929-4e1a-9c5b-50b210ae130d].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/c9f3737a-5929-4e1a-9c5b-50b210ae130d?project=934903580331 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "c9f3737a-5929-4e1a-9c5b-50b210ae130d"

FETCHSOURCE
Fetching storage object: gs://hybrid-vertex_cloudbuild/source/1700046469.187503-dcbf29813b914ce4b252c64e7e879c14.tgz#1700046469844197
Copying gs://hybrid-vertex_cloudbuild/source/1700046469.187503-dcbf29813b914ce4b252c64e7e879c14.tgz#1700046469844197...
/ [1 files][125.6 KiB/125.6 KiB]                                                
Operation completed over 1 objects/12

## (Optional) Build Image Locally

In [23]:
ARTIFACTS_DIR = 'gs://rec-bandits-v2-hybrid-vertex-bucket/scale-comparisons-v1/run-20231107-161324/artifacts'

In [24]:
# ! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet
print(f"gcloud auth configure-docker {REGION}-docker.pkg.dev")

gcloud auth configure-docker us-central1-docker.pkg.dev


In [25]:
print("copy these commands into terminal:\n")
print(f"virtualenv vertex_env")
print(f"source vertex_env/bin/activate")

copy these commands into terminal:

virtualenv vertex_env
source vertex_env/bin/activate


In [27]:
# # set variables if running in terminal
print("copy these commands into terminal:\n")
print(f"cd {SERVING_DIR}")
print(f"export REMOTE_IMAGE_NAME={REMOTE_IMAGE_NAME}")
print(f"export DOCKERNAME={DOCKERNAME_02_PRED}")
print(f"export ARTIFACTS_DIR={ARTIFACTS_DIR}")
print(f"export IMAGE_NAME_02_PRED={IMAGE_NAME_02_PRED}")

print(f"docker build --no-cache -t $REMOTE_IMAGE_NAME -f ./$DOCKERNAME .")
# print(f"docker build --no-cache -t $REMOTE_IMAGE_NAME -f ./src/pred/$DOCKERNAME .")

copy these commands into terminal:

cd serve
export REMOTE_IMAGE_NAME=us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/pred-perarm-feats-02e
export DOCKERNAME=Dockerfile_predict_mab_02e
export ARTIFACTS_DIR=gs://rec-bandits-v2-hybrid-vertex-bucket/scale-comparisons-v1/run-20231107-161324/artifacts
export IMAGE_NAME_02_PRED=pred-perarm-feats-02e
docker build --no-cache -t $REMOTE_IMAGE_NAME -f ./$DOCKERNAME .


In [39]:
# ### push the container to registry

print("copy this command into terminal:\n")
print(f"docker push $REMOTE_IMAGE_NAME")

# !docker push $REMOTE_IMAGE_NAME

copy this command into terminal:

docker push $REMOTE_IMAGE_NAME


In [28]:
# ! docker rm local-fastapi 2>/dev/null
docker run -t -d -p 7080:7080 \
    --name=local-fastapi \
    -e AIP_HTTP_PORT=7080 \
    -e AIP_HEALTH_ROUTE=/health \
    -e AIP_PREDICT_ROUTE=/predict \
    -e AIP_STORAGE_URI=$ARTIFACTS_DIR \
    -e AIP_MODEL_DIR=$ARTIFACTS_DIR \
    $REMOTE_IMAGE_NAME
! docker container ls

429bab9f02d672e92bc7a319f99ff7070c0e2af8e4e5e57a0797cb931a65bff8
CONTAINER ID   IMAGE                                                                                        COMMAND                  CREATED                  STATUS                  PORTS                                                 NAMES
429bab9f02d6   us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/pred-perarm-feats-02e   "/entrypoint.sh sh -…"   Less than a second ago   Up Less than a second   0.0.0.0:7080->7080/tcp, :::7080->7080/tcp, 8080/tcp   pred-perarm-feats-02e


In [33]:
# ! docker stop $IMAGE_NAME_02_PRED

Error response from daemon: No such container: pred-perarm-feats-02e


In [57]:
# print("copy this command into terminal:\n")
# COMMAND=f'''
# docker run -d -p 80:8080 --name=$IMAGE_NAME_02_PRED -e AIP_HTTP_PORT=8080 -e AIP_HEALTH_ROUTE=/health -e AIP_PREDICT_ROUTE=/predict -e AIP_STORAGE_URI=$ARTIFACTS_DIR $REMOTE_IMAGE_NAME
# '''
# print(COMMAND)

copy this command into terminal:


docker run -d -p 80:8080 --name=$IMAGE_NAME_02_PRED -e AIP_HTTP_PORT=8080 -e AIP_HEALTH_ROUTE=/health -e AIP_PREDICT_ROUTE=/predict -e AIP_STORAGE_URI=$ARTIFACTS_DIR $REMOTE_IMAGE_NAME



In [30]:
# print("copy this command into terminal:\n")
# print(f"curl localhost/health")

# ! curl localhost/health

! curl http://localhost:7080/health

curl: (7) Failed to connect to localhost port 7080: Connection refused


In [40]:
print("copy this command into terminal:\n")
# print(f"docker kill $IMAGE_NAME_02_PRED")
print(f"docker kill $REMOTE_IMAGE_NAME")

copy this command into terminal:

docker kill $REMOTE_IMAGE_NAME


In [37]:
print("copy this command into terminal:\n")
# print(f"docker rm $IMAGE_NAME_02_PRED")
print(f"docker rm $REMOTE_IMAGE_NAME")

copy this command into terminal:

docker rm $REMOTE_IMAGE_NAME


# Debugging pred container

## tmp - Vertex AI Local Predict

* [source](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Triton_PyTorch_Local_Prediction.ipynb)

In [82]:
CREDENTIALS_FILE = "./credentials.json"
# CREDENTIALS_FILE = "/home/jupyter/.config/gcloud/application_default_credentials.json"

In [83]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account=$VERTEX_SA

created key [7afabb0cfe6a9d4edbec8bcb0854f45f460ac3ed] of type [json] as [./credentials.json] for [934903580331-compute@developer.gserviceaccount.com]


In [61]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel(
    serving_container_image_uri=REMOTE_IMAGE_NAME,
    serving_container_predict_route="/predict",
    serving_container_health_route="/health",
    serving_container_ports=[8080],
)

You can check out the serving container spec of the image.

In [62]:
local_model.get_serving_container_spec()

image_uri: "us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/pred-perarm-feats-03"
ports {
  container_port: 8080
}
predict_route: "/predict"
health_route: "/health"

In [64]:
# local_model.

In [65]:
INPUT_FILE = "pred_instances.json"

In [66]:
%%writefile $INPUT_FILE
{
    "instances": [
        {
            'bucketized_user_age': 25.0,
            'movie_genres':[4],
            'movie_id': '211',
            'timestamp':874948475,
            'user_id': '346',
            'user_occupation_text':'other',
            'user_rating': 4.,
        }
    ]
}

Writing pred_instances.json


In [84]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=ARTIFACTS_DIR,
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

In [85]:
print(predict_response, predict_response.content)

<Response [500]> b'Internal Server Error'


In [86]:
print(health_check_response, health_check_response.content)

<Response [200]> b'{}'


In [87]:
local_endpoint.print_container_logs(show_all=True)

### test local predict from remote image

In [72]:
local_model_ar = local_model.copy_image(
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE_NAME_02_PRED}"
)

local_model_ar.get_serving_container_spec()

/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


image_uri: "us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/pred-perarm-feats-03"
ports {
  container_port: 8080
}
predict_route: "/predict"
health_route: "/health"

In [ ]:
# local_model_ar.

## tmp - jw docker predict

* [source](https://github.com/jswortz/google-vertex-cpr-pipeline-example/blob/master/SDK_Pytorch_Custom_Predict-MLOps-Edition.ipynb)

In [46]:
base64_test = base64.b64encode(b'211').decode("utf-8")
base64_test

'MjEx'

In [57]:
import requests
import json
from pprint import pprint
import base64

TEST_INSTANCE = [
    {
        'bucketized_user_age': 25.0,
        'movie_genres':[4],
        'movie_id': '211',
        'timestamp':874948475,
        'user_id': '346',
        'user_occupation_text':'other',
        'user_rating': 4.,
    }
]

# TEST_INSTANCE = [{
#     'bucketized_user_age': 25.,
#     'movie_genres':[4],
#     'movie_id': base64.b64encode(b'211').decode("utf-8"),
#     'timestamp':[874948475],
#     'user_id': base64.b64encode(b'346').decode("utf-8"),
#     'user_occupation_text':base64.b64encode(b'other').decode("utf-8"),
#     'user_rating': [4.],
# }]
pprint(TEST_INSTANCE)

[{'bucketized_user_age': 25.0,
  'movie_genres': [4],
  'movie_id': '211',
  'timestamp': 874948475,
  'user_id': '346',
  'user_occupation_text': 'other',
  'user_rating': 4.0}]


In [76]:
json_instance = json.dumps({"instances": TEST_INSTANCE})

print("Google formatted instance: ", json_instance)

Google formatted instance:  {"instances": [{"bucketized_user_age": 25.0, "movie_genres": [4], "movie_id": "211", "timestamp": 874948475, "user_id": "346", "user_occupation_text": "other", "user_rating": 4.0}]}


In [42]:
# requests.post('http://localhost/predict', data=json_instance).json()

### tmp - http response body

In [58]:
from google.api import httpbody_pb2

http_body = httpbody_pb2.HttpBody(
    data=json.dumps(TEST_INSTANCE).encode("utf-8"),
    content_type="application/json",
)

http_body

content_type: "application/json"
data: "[{\"bucketized_user_age\": 25.0, \"movie_genres\": [4], \"movie_id\": \"211\", \"timestamp\": 874948475, \"user_id\": \"346\", \"user_occupation_text\": \"other\", \"user_rating\": 4.0}]"

In [78]:
# requests.post('http://localhost/predict', data=http_body).json()

In [33]:
contents = json.loads(json_instance)
contents

{'instances': {'bucketized_user_age': [25.0],
  'movie_genres': [[4]],
  'movie_id': ['211'],
  'timestamp': [874948475],
  'user_id': ['346'],
  'user_occupation_text': ['other'],
  'user_rating': [4.0]}}

# Register and Deploy policy

In [94]:
from google.cloud import aiplatform as vertex_ai
from google.cloud import storage

from tf_agents.policies import py_tf_eager_policy

In [29]:
print(f"DOCKERNAME_02_PRED = {DOCKERNAME_02_PRED}")
print(f"REPOSITORY         = {REPOSITORY}")
print(f"IMAGE_NAME_02_PRED = {IMAGE_NAME_02_PRED}")
print(f"IMAGE_URI_02_PRED  = {IMAGE_URI_02_PRED}")
print(f"REMOTE_IMAGE_NAME  = {REMOTE_IMAGE_NAME}")

DOCKERNAME_02_PRED = Dockerfile_predict_mab_02e
REPOSITORY         = rl-movielens-rec-bandits-v2
IMAGE_NAME_02_PRED = pred-perarm-feats-02e
IMAGE_URI_02_PRED  = gcr.io/hybrid-vertex/pred-perarm-feats-02e
REMOTE_IMAGE_NAME  = us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-rec-bandits-v2/pred-perarm-feats-02e


### Upload to Vertex AI Model Registry

In [30]:
AGENT_TYPE = "epsGreedy"
# ARTIFACTS_DIR = 'gs://rec-bandits-v2-hybrid-vertex-bucket/scale-comparisons-v1/run-20231107-161324/artifacts'
ARTIFACTS_DIR = "gs://rec-bandits-v2-hybrid-vertex-bucket/02-scale-compare-v2/run-20231115-094131/artifacts"

In [95]:
test_policy = py_tf_eager_policy.SavedModelPyTFEagerPolicy(
    ARTIFACTS_DIR, load_specs_from_pbtxt=True
)

In [98]:
# test_policy.

In [31]:
uploaded_policy = vertex_ai.Model.upload(
    display_name=f'trained-{AGENT_TYPE}-v1',
    artifact_uri=ARTIFACTS_DIR,
    serving_container_image_uri=f"{IMAGE_URI_02_PRED}:latest",
    # serving_container_image_uri=f"{REMOTE_IMAGE_NAME}:latest",
    serving_container_predict_route="/predict",
    serving_container_health_route="/health",
    sync=True,
)

print(f"display_name    : {uploaded_policy.display_name}")
print(f"uploaded_policy : {uploaded_policy}")

Creating Model
Create Model backing LRO: projects/934903580331/locations/us-central1/models/3417658172104507392/operations/8239509722147848192
Model created. Resource name: projects/934903580331/locations/us-central1/models/3417658172104507392@1
To use this Model in another session:
model = aiplatform.Model('projects/934903580331/locations/us-central1/models/3417658172104507392@1')
display_name    : trained-epsGreedy-v1
uploaded_policy : <google.cloud.aiplatform.models.Model object at 0x7f7d501dc9d0> 
resource name: projects/934903580331/locations/us-central1/models/3417658172104507392


## Create endpoint

In [32]:
# endpoint = vertex_ai.Endpoint.create(
#     display_name=f'endpoint-{AGENT_TYPE}-v2',
#     project=PROJECT_ID,
#     location=LOCATION,
#     sync=True,
# )

# endpoint = vertex_ai.Endpoint('211442683091091456')
endpoint = vertex_ai.Endpoint('3091072432329457664')

print(f"display_name : {endpoint.display_name}")
print(f"endpoint     : {endpoint}")

display_name : endpoint-epsGreedy-v2
endpoint     : <google.cloud.aiplatform.models.Endpoint object at 0x7f7d807d16f0> 
resource name: projects/934903580331/locations/us-central1/endpoints/3091072432329457664


## Deploy Registered policy to endpoint

In [33]:
deployed_policy = uploaded_policy.deploy(
    endpoint=endpoint,
    deployed_model_display_name=f'deployed-{AGENT_TYPE}-v1',
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=1,
    accelerator_type=None,
    accelerator_count=0,
    sync=True,
)

print(f"display_name    : {deployed_policy.display_name}\n")
print(f"deployed_policy : {deployed_policy}")

deployed_policy

Deploying model to Endpoint : projects/934903580331/locations/us-central1/endpoints/3091072432329457664
Deploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/3091072432329457664/operations/516962261114290176
Endpoint model deployed. Resource name: projects/934903580331/locations/us-central1/endpoints/3091072432329457664
display_name    : endpoint-epsGreedy-v2

deployed_policy : <google.cloud.aiplatform.models.Endpoint object at 0x7f7d807d16f0> 
resource name: projects/934903580331/locations/us-central1/endpoints/3091072432329457664


resource name: projects/934903580331/locations/us-central1/endpoints/3091072432329457664

### Test endpoint

In [215]:
TEST_INSTANCE = {"observation": [list(np.ones(20)) for _ in range(8)]}
TEST_INSTANCE["observation"][1]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [36]:
# !pwd

'rec-bandits-v2-hybrid-vertex-bucket'

In [50]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

# tensorflow
import tensorflow as tf

from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

# this repo
from src.per_arm_rl import data_utils

In [142]:
SPLIT = "val"
eval_batch_size = 1
NUM_EVAL_STEPS = 20

val_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_GCS_PREFIX}/{SPLIT}'):
    if '.tfrecord' in blob.name:
        val_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
        
val_dataset = tf.data.TFRecordDataset(val_files)
val_dataset = val_dataset.map(data_utils.parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)

# eval dataset
eval_ds = val_dataset.batch(eval_batch_size)

# if NUM_EVAL_STEPS > 0:
#     eval_ds = eval_ds.take(NUM_EVAL_STEPS)

# eval_ds = eval_ds.prefetch(tf.data.AUTOTUNE)

eval_ds

<_BatchDataset element_spec={'bucketized_user_age': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'movie_genres': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'timestamp': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_occupation_text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}>

In [201]:
data_list = [] 

for i in range(2):
    
    iterator = iter(val_dataset.batch(1))
    data = next(iterator)
    data_list.append(data)

data_list

[{'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>,
  'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
  'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'211'], dtype=object)>,
  'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([874948475])>,
  'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>,
  'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other'], dtype=object)>,
  'user_rating': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>},
 {'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>,
  'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
  'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'211'], dtype=object)>,
  'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([874948475])>,
  'user_id': <tf.Tens

In [203]:
len(data_list)

2

In [204]:
global_fets_list = []
for instance in data_list:
    global_feat_infer = embs._get_global_context_features(instance)
    arm_feat_infer = embs._get_per_arm_features(instances)
    
global_fets_list

[array([[-0.02284321, -0.04486708,  0.01871774, -0.03706668, -0.03948395,
          0.00815228,  0.04989055, -0.00984588, -0.03338911, -0.00469216,
         -0.01469489,  0.01931005,  0.00450007,  0.04033008, -0.04139885,
          0.03369157, -0.02654978,  0.02733845,  0.02188141,  0.03641399,
         -0.01575927, -0.04570995, -0.04994596, -0.04506509,  0.03897073,
         -0.02349632, -0.01979375, -0.00072138,  0.00268871, -0.04270414,
          0.04827886,  0.00287588, -0.02039087, -0.00033497,  0.01710043,
          0.04911342,  0.03376654,  0.03906982, -0.02361402, -0.03907317,
         -0.00081597, -0.02704746, -0.03288104, -0.01972567, -0.02979683,
          0.01936915, -0.02871168,  0.0135267 , -0.03308724,  0.01739838,
         -0.0053169 , -0.0357516 ,  0.0214668 , -0.00639789,  0.03177608,
          0.01960037,  0.03402347,  0.02833668,  0.03471385,  0.01466086,
          0.01016812, -0.01258299, -0.04159774,  0.04512725, -0.01855729,
          0.00272835,  0.02515319,  0.

In [206]:
len(global_fets_list)

2

In [208]:
len(global_fets_list[0][0])

256

In [167]:
for tensor_dict in eval_ds.take(1):
    td_keys = tensor_dict.keys()
    list_dict = {}
    for k in td_keys:
        val = tensor_dict[k].numpy()
        # val = val.item().tolist()
        val = val.tolist()
        # list_dict.update({k: tensor_dict[k].numpy()})
        list_dict.update({k: val})
    print(list_dict)

{'bucketized_user_age': [25.0], 'movie_genres': [[4]], 'movie_id': [b'211'], 'timestamp': [874948475], 'user_id': [b'346'], 'user_occupation_text': [b'other'], 'user_rating': [4.0]}


In [182]:
instances = {"instances": list_dict}
instances

{'instances': {'bucketized_user_age': [25.0],
  'movie_genres': [[4]],
  'movie_id': [b'211'],
  'timestamp': [874948475],
  'user_id': [b'346'],
  'user_occupation_text': [b'other'],
  'user_rating': [4.0]}}

In [189]:
jt_iter_test = {
    'bucketized_user_age': [25.0],
    'movie_genres': [4],
    'movie_id': ['211'],
    'timestamp': [874948475],
    'user_id': '346',
    'user_occupation_text': ['other'],
    'user_rating': [4.0]
}

instances = {"instances": jt_iter_test}
instances

{'instances': {'bucketized_user_age': [25.0],
  'movie_genres': [4],
  'movie_id': ['211'],
  'timestamp': [874948475],
  'user_id': '346',
  'user_occupation_text': ['other'],
  'user_rating': [4.0]}}

In [190]:
deployed_policy.predict(
    instances=jt_iter_test,
)

InternalServerError: 500 Internal Server Error

### local reproduce

In [122]:
import pickle as pkl

LOCAL_VOCAB_FILENAME = './src/pred/vocab_dict.pkl'

filehandler = open(f"{LOCAL_VOCAB_FILENAME}", 'rb')
vocab_dict = pkl.load(filehandler)
filehandler.close()

In [123]:
from src.pred import emb_features_pred as emb_features
# from src.perarm_features import emb_features as emb_features

In [124]:
embs = emb_features.EmbeddingModel(
    vocab_dict = vocab_dict,
    num_oov_buckets = 1,
    global_emb_size = 64,
    mv_emb_size = 32,
)

embs

In [125]:
x

{'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>,
 'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'211'], dtype=object)>,
 'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([874948475])>,
 'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>,
 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other'], dtype=object)>,
 'user_rating': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>}

In [135]:
val_iter = val_dataset.as_numpy_iterator()
iter_x = val_iter.next()

iter_x # TypeError: Inputs to a layer should be tensors. Got 'b'346'' (of type <class 'bytes'>) as input for layer 'model'.

{'bucketized_user_age': 25.0,
 'movie_genres': array([4]),
 'movie_id': b'211',
 'timestamp': 874948475,
 'user_id': b'346',
 'user_occupation_text': b'other',
 'user_rating': 4.0}

In [137]:
jt_iter_test = {
    'bucketized_user_age': [25.0],
    'movie_genres': [4],
    'movie_id': ['211'],
    'timestamp': [874948475],
    'user_id': ['346'],
    'user_occupation_text': ['other'],
    'user_rating': [4.0]
}

In [216]:
data

{'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>,
 'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'211'], dtype=object)>,
 'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([874948475])>,
 'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>,
 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other'], dtype=object)>,
 'user_rating': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>}

In [187]:
# global_feat_infer = embs._get_global_context_features([[jt_iter_test]])

global_feat_infer = embs._get_global_context_features(data)
global_feat_infer

array([[-0.02284321, -0.04486708,  0.01871774, -0.03706668, -0.03948395,
         0.00815228,  0.04989055, -0.00984588, -0.03338911, -0.00469216,
        -0.01469489,  0.01931005,  0.00450007,  0.04033008, -0.04139885,
         0.03369157, -0.02654978,  0.02733845,  0.02188141,  0.03641399,
        -0.01575927, -0.04570995, -0.04994596, -0.04506509,  0.03897073,
        -0.02349632, -0.01979375, -0.00072138,  0.00268871, -0.04270414,
         0.04827886,  0.00287588, -0.02039087, -0.00033497,  0.01710043,
         0.04911342,  0.03376654,  0.03906982, -0.02361402, -0.03907317,
        -0.00081597, -0.02704746, -0.03288104, -0.01972567, -0.02979683,
         0.01936915, -0.02871168,  0.0135267 , -0.03308724,  0.01739838,
        -0.0053169 , -0.0357516 ,  0.0214668 , -0.00639789,  0.03177608,
         0.01960037,  0.03402347,  0.02833668,  0.03471385,  0.01466086,
         0.01016812, -0.01258299, -0.04159774,  0.04512725, -0.01855729,
         0.00272835,  0.02515319,  0.01827284,  0.0

In [117]:
np.array(x['user_id'].numpy())

array([b'346'], dtype=object)

#### Raw Predict

* see [example](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/ef8b70db32813b8a2f128ab5ef1d170aea739e7f/notebooks/official/prediction/get_started_with_raw_predict.ipynb)

In [238]:
# jt_raw_test = {
#     'bucketized_user_age': tf.constant([25.0], dtype=tf.float32),
#     'movie_genres': tf.constant([4], dtype=tf.int64),
#     'movie_id': tf.constant(['211'], dtype=tf.string),
#     'timestamp': tf.constant([874948475], dtype=tf.int64),
#     'user_id': tf.constant(['346'], dtype=tf.string),
#     'user_occupation_text': tf.constant(['other'], dtype=tf.string),
#     'user_rating': tf.constant([4.0], dtype=tf.float32)
# }

# jt_raw_test = {
#     'bucketized_user_age': [25.0],
#     'movie_genres': [[4]],
#     'movie_id': [b'211'],
#     'timestamp': [874948475],
#     'user_id': ['346'],
#     'user_occupation_text': [b'other'],
#     'user_rating': [4.0]
# }

jt_raw_test = data#['user_id']

jt_raw_test

{'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>,
 'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'211'], dtype=object)>,
 'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([874948475])>,
 'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>,
 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other'], dtype=object)>,
 'user_rating': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>}

In [240]:
# jt_raw_test.numpy().tolist()

tes = [25.0]

jt_raw_test['user_id'].numpy().tolist()

TEST_INSTANCE = {
    "instances":
    {
        'bucketized_user_age': jt_raw_test['bucketized_user_age'].numpy().tolist(),
        'movie_genres': jt_raw_test['movie_genres'].numpy().tolist(),
        'movie_id': jt_raw_test['movie_id'].numpy().tolist(),
        'timestamp': jt_raw_test['timestamp'].numpy().tolist(),
        'user_id': jt_raw_test['user_id'].numpy().tolist(),
        'user_occupation_text': jt_raw_test['user_occupation_text'].numpy().tolist(),
        'user_rating': jt_raw_test['user_rating'].numpy().tolist()
    }
}
TEST_INSTANCE

{'instances': {'bucketized_user_age': [25.0],
  'movie_genres': [[4]],
  'movie_id': [b'211'],
  'timestamp': [874948475],
  'user_id': [b'346'],
  'user_occupation_text': [b'other'],
  'user_rating': [4.0]}}

In [242]:
TEST_INSTANCE['instances']

{'bucketized_user_age': [25.0],
 'movie_genres': [[4]],
 'movie_id': [b'211'],
 'timestamp': [874948475],
 'user_id': [b'346'],
 'user_occupation_text': [b'other'],
 'user_rating': [4.0]}

In [244]:
tf.constant(TEST_INSTANCE['instances']['user_id'])

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>

In [218]:
embs.user_id_model(jt_raw_test['user_id'])

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[-0.02284321, -0.04486708,  0.01871774, -0.03706668, -0.03948395,
         0.00815228,  0.04989055, -0.00984588, -0.03338911, -0.00469216,
        -0.01469489,  0.01931005,  0.00450007,  0.04033008, -0.04139885,
         0.03369157, -0.02654978,  0.02733845,  0.02188141,  0.03641399,
        -0.01575927, -0.04570995, -0.04994596, -0.04506509,  0.03897073,
        -0.02349632, -0.01979375, -0.00072138,  0.00268871, -0.04270414,
         0.04827886,  0.00287588, -0.02039087, -0.00033497,  0.01710043,
         0.04911342,  0.03376654,  0.03906982, -0.02361402, -0.03907317,
        -0.00081597, -0.02704746, -0.03288104, -0.01972567, -0.02979683,
         0.01936915, -0.02871168,  0.0135267 , -0.03308724,  0.01739838,
        -0.0053169 , -0.0357516 ,  0.0214668 , -0.00639789,  0.03177608,
         0.01960037,  0.03402347,  0.02833668,  0.03471385,  0.01466086,
         0.01016812, -0.01258299, -0.04159774,  0.04512725]],
      dtype=f

In [243]:
embs.user_id_model(tf.constant(TEST_INSTANCE['instances']['user_id']))

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[-0.02284321, -0.04486708,  0.01871774, -0.03706668, -0.03948395,
         0.00815228,  0.04989055, -0.00984588, -0.03338911, -0.00469216,
        -0.01469489,  0.01931005,  0.00450007,  0.04033008, -0.04139885,
         0.03369157, -0.02654978,  0.02733845,  0.02188141,  0.03641399,
        -0.01575927, -0.04570995, -0.04994596, -0.04506509,  0.03897073,
        -0.02349632, -0.01979375, -0.00072138,  0.00268871, -0.04270414,
         0.04827886,  0.00287588, -0.02039087, -0.00033497,  0.01710043,
         0.04911342,  0.03376654,  0.03906982, -0.02361402, -0.03907317,
        -0.00081597, -0.02704746, -0.03288104, -0.01972567, -0.02979683,
         0.01936915, -0.02871168,  0.0135267 , -0.03308724,  0.01739838,
        -0.0053169 , -0.0357516 ,  0.0214668 , -0.00639789,  0.03177608,
         0.01960037,  0.03402347,  0.02833668,  0.03471385,  0.01466086,
         0.01016812, -0.01258299, -0.04159774,  0.04512725]],
      dtype=f

In [219]:
np.array(embs.user_id_model(jt_raw_test['user_id']).numpy())

array([[-0.02284321, -0.04486708,  0.01871774, -0.03706668, -0.03948395,
         0.00815228,  0.04989055, -0.00984588, -0.03338911, -0.00469216,
        -0.01469489,  0.01931005,  0.00450007,  0.04033008, -0.04139885,
         0.03369157, -0.02654978,  0.02733845,  0.02188141,  0.03641399,
        -0.01575927, -0.04570995, -0.04994596, -0.04506509,  0.03897073,
        -0.02349632, -0.01979375, -0.00072138,  0.00268871, -0.04270414,
         0.04827886,  0.00287588, -0.02039087, -0.00033497,  0.01710043,
         0.04911342,  0.03376654,  0.03906982, -0.02361402, -0.03907317,
        -0.00081597, -0.02704746, -0.03288104, -0.01972567, -0.02979683,
         0.01936915, -0.02871168,  0.0135267 , -0.03308724,  0.01739838,
        -0.0053169 , -0.0357516 ,  0.0214668 , -0.00639789,  0.03177608,
         0.01960037,  0.03402347,  0.02833668,  0.03471385,  0.01466086,
         0.01016812, -0.01258299, -0.04159774,  0.04512725]],
      dtype=float32)

In [246]:
deployed_policy.predict(
    instances=[TEST_INSTANCE],
)

InternalServerError: 500 Internal Server Error

In [164]:
import json

from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

In [179]:
# TEST_INSTANCE = {
#     "instances": [
#         {
#             'bucketized_user_age': [25.0],
#             'movie_genres': [4],
#             'movie_id': ['211'],
#             'timestamp': [874948475],
#             'user_id': [['346']],
#             'user_occupation_text': ['other'],
#             'user_rating': [4.0]
#         }
#     ]
# }

# TEST_INSTANCE = {
#     "instances":
#     {
#         'bucketized_user_age': [25.0],
#         'movie_genres': [4],
#         'movie_id': ['211'],
#         'timestamp': [874948475],
#         'user_id': ['346'],
#         'user_occupation_text': ['other'],
#         'user_rating': [4.0]
#     }
# }

# TEST_INSTANCE = {
#     "instances": jt_raw_test
# }

TEST_INSTANCE = {
    "instances": [
        {
            'bucketized_user_age': tf.constant([25.0], dtype=tf.float32),
            'movie_genres': tf.constant([4], dtype=tf.int64),
            'movie_id': tf.constant(['211'], dtype=tf.string),
            'timestamp': tf.constant([874948475], dtype=tf.int64),
            'user_id': tf.constant(['346'], dtype=tf.string),
            'user_occupation_text': tf.constant(['other'], dtype=tf.string),
            'user_rating': tf.constant([4.0], dtype=tf.float32)
        }
    ]
}

http_body = httpbody_pb2.HttpBody(
    data=json.dumps(TEST_INSTANCE).encode("utf-8"),
    content_type="application/json",
)

req = aiplatform_v1.RawPredictRequest(
    http_body=http_body, endpoint=endpoint.resource_name
)

req

TypeError: Object of type EagerTensor is not JSON serializable

In [159]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}

pred_client = vertex_ai.gapic.PredictionServiceClient(client_options=client_options)
pred_client

In [160]:
response = pred_client.raw_predict(req)
print(response)

InternalServerError: 500 Internal Server Error

In [153]:
deployed_policy.predict(
    instances=data,
)

ValueError: Unable to coerce value: <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>

In [46]:
for x in eval_ds.take(1):

    deployed_policy.predict(
        instances=[x],
    )

ValueError: Unable to coerce value: <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>

In [45]:
x

{'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>,
 'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'211'], dtype=object)>,
 'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([874948475])>,
 'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>,
 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other'], dtype=object)>,
 'user_rating': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>}

In [143]:
eval_ds

<_BatchDataset element_spec={'bucketized_user_age': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'movie_genres': TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'timestamp': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_occupation_text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}>

In [ ]:
for i in range(1):
    
    iterator = iter(train_dataset.batch(1))
    data = next(iterator)

data

In [145]:
for x in eval_ds.take(1):

    deployed_policy.predict(
        instances=[x],
    )

ValueError: Unable to coerce value: <tf.Tensor: shape=(1,), dtype=float32, numpy=array([25.], dtype=float32)>

## Clean up

In [34]:
# endpoint.delete(force=True)

# uploaded_policy.delete()

In [ ]:
import os
import sys
import logging
import numpy as np
import pickle as pkl
from typing import Dict, Any, Tuple

logging.disable(logging.WARNING)

# google cloud
from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.utils import prediction_utils
from google.cloud import storage

# tensorflow
import tensorflow as tf
import tf_agents
from tf_agents.policies import py_tf_eager_policy
from tf_agents.trajectories import time_step as ts

# this repo
sys.path.extend([f'./{name}' for name in os.listdir(".") if os.path.isdir(name)])

import data_config as data_config
import emb_features_pred as emb_features
import reward_factory as reward_factory

os.environ["PROJECT_ID"] = data_config.PROJECT_ID

# ==================================
# get trajectory step for prediction
# ==================================
def _get_pred_step(feature, reward_np):
    
    infer_step = ts.TimeStep(
        tf.constant(ts.StepType.FIRST, dtype=tf.int32, shape=[],name='step_type'),
        tf.constant(reward_np, dtype=tf.float32, shape=[], name='reward'),
        tf.constant(1.0, dtype=tf.float32, shape=[], name='discount'),
        feature
    )
    
    return infer_step

# ==================================
# prediction logic
# ==================================
class BanditPolicyPredictor(Predictor):
    
    """
    Interface of the Predictor class for Custom Prediction Routines.
    
    The Predictor is responsible for the ML logic for processing a prediction request.
    
    Specifically, the Predictor must define:
        (1) How to load all model artifacts used during prediction into memory.
        (2) The logic that should be executed at predict time.
    
    When using the default PredictionHandler, the Predictor will be invoked as follows:
    
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    
    """
    
    def __init__(self):
        
        self._local_vocab_filename = "./vocab_dict.pkl"
        self._num_oov_buckets = data_config.NUM_OOV_BUCKETS
        self._global_embedding_size = data_config.GLOBAL_EMBEDDING_SIZE
        self._mv_embedding_size = data_config.MV_EMBEDDING_SIZE
        return
        
    def load(self, artifacts_uri: str):
        """
        Loads trained policy dir & vocabulary
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
                has `vocab_dict.pkl` and `artifacts/` directory
        
        """
        prediction_utils.download_model_artifacts(artifacts_uri)
        
        # init deploy policy
        self._deployment_policy = py_tf_eager_policy.SavedModelPyTFEagerPolicy(
            'artifacts', load_specs_from_pbtxt=True
        )
        
        # load vocab dict
        filehandler = open(f"{self._local_vocab_filename}", 'rb')
        self._vocab_dict = pkl.load(filehandler)
        filehandler.close()
        
        # only if no custom preprocessor is defined
        # self._preprocessor = preprocessor
        
    def preprocess(self, prediction_input: Dict): # -> Tuple[Dict, float]:
        """
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.        
        """
        # inputs = super().preprocess(prediction_input)
        
        dummy_arm = tf.zeros([1, data_config.PER_ARM_DIM], dtype=tf.float32)
        
        batch_size = len(prediction_input["instances"])
        assert batch_size == 1, 'prediction batch_size must be == 1'
        
        self._embs = emb_features.EmbeddingModel(
            vocab_dict = self._vocab_dict,
            num_oov_buckets = self._num_oov_buckets,
            global_emb_size = self._global_embedding_size,
            mv_emb_size = self._mv_embedding_size,
        )
        
        # preprocess example
        rebuild_ex = {}

        for x in prediction_input["instances"]:
            rebuild_ex['bucketized_user_age'] = tf.constant([x["bucketized_user_age"]], dtype=tf.float32)
            rebuild_ex['movie_genres'] = tf.constant([x["movie_genres"]], dtype=tf.int64)
            rebuild_ex['movie_id'] = tf.constant([x["movie_id"]], dtype=tf.string)
            rebuild_ex['timestamp'] = tf.constant([x["timestamp"]], dtype=tf.int64)
            rebuild_ex['user_id'] = tf.constant([x["user_id"]], dtype=tf.string)
            rebuild_ex['user_occupation_text'] = tf.constant([x["user_occupation_text"]], dtype=tf.string)
            rebuild_ex['user_rating'] = tf.constant([x["user_rating"]], dtype=tf.float32)
        
        global_feat_infer = self._embs._get_global_context_features(rebuild_ex)
        logging.info(f'global_feat_infer: {global_feat_infer}')          # tmp - debugging
        
        arm_feat_infer = self._embs._get_per_arm_features(rebuild_ex)    # tmp - debugging
        logging.info(f'arm_feat_infer: {arm_feat_infer}')
    
        rewards = reward_factory._get_rewards(rebuild_ex)
        logging.info(f'rewards: {rewards}')                              # tmp - debugging
        
        actual_reward = rewards.numpy()[0]
        logging.info(f'actual_reward: {actual_reward}')                  # tmp - debugging
        
        arm_feat_infer = tf.reshape(arm_feat_infer, [1, data_config.PER_ARM_DIM])
        concat_arm = tf.concat([arm_feat_infer, dummy_arm], axis=0)      # tmp - debugging
        
        # flatten global
        flat_global_infer = tf.reshape(global_feat_infer, [data_config.GLOBAL_DIM])
        feature = {'global': flat_global_infer, 'per_arm': concat_arm}
        logging.info(f'feature: {feature}')                              # tmp - debugging
        
        trajectory_step = _get_pred_step(feature, actual_reward)
        logging.info(f'trajectory_step: {trajectory_step}')
        
        # prediction = self._deployment_policy.action(trajectory_step)
        
        return trajectory_step
    
    def predict(self, instances) -> Dict:
        """
        Performs prediction.
        """
        
        prediction = self._deployment_policy.action(instances) # trajectory_step
        
        return {"predictions": prediction}
        

#     def postprocess(self, prediction_results: Any) -> Any:
#         """ 
#         TODO:
#              Convert predictions to item IDs
#         """
        
#         pass

**Finished**